In [1]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
from scipy.io import loadmat


directory = "/scratch/ebron/ADNI/"
pmap_matfile = os.path.join(directory, "Pmaps", "pmap_t1.mat")
mask_filename = os.path.join(directory, 'Template_space', 'brain_mask_in_template_space.nii.gz')


In [11]:
mask = nib.load(mask_filename)
mask_image = mask.get_data()
print(mask.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 182 218 182   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [-1.  1.  1.  1.  0.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : aligned
sform_code      : scanner
quatern_b       : -0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [-1. -0.  0. 90.]
srow_y

In [12]:
# Load pmap from .mat file
pmap_struct = loadmat(pmap_matfile)
pmap = [p[0] for p in pmap_struct['pmap']]
del pmap_struct

# Scale
pmap = np.array(pmap)*1000
pmap[pmap<1] = 1
pmap = list(pmap)

# Put in image
pmap_image = mask_image.copy()
pmap_image[mask_image==1] = pmap 

# Remove single voxels
pmap_binary = np.where(pmap_image<=50, 1, 0)
from scipy.ndimage.morphology import binary_opening
pmap_binary = binary_opening(pmap_binary)

pmap_image = np.multiply(pmap_image, pmap_binary)
pmap_image[pmap_image==0] = 1000


# Save as nifti
pmap_header = mask.header.copy()
pmap_header.set_data_dtype(np.uint16)
pmap_nii = nib.Nifti1Image(pmap_image, mask.affine, pmap_header)
pmap_nii.to_filename(os.path.join(directory, 'Pmaps', 'Pmap_analytic_margin_nosinglevoxels_t1.nii.gz'))


ValueError: NumPy boolean array indexing assignment cannot assign 100000 input values to the 1762276 output values where the mask is true